In [13]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import torch

In [15]:
quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

device_map = {
    "transformer.word_embeddings": 0,
    "transformer.word_embeddings_layernorm": 0,
    "lm_head": "cpu",
    "transformer.h": 0,
    "transformer.ln_f": 0
}

model_name = "chavinlo/alpaca-native"

max_memory_mapping = {0: "5GB"}

tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map=device_map, quantization_config=quantization_config, max_memory=max_memory_mapping)

Loading checkpoint shards:   0%|          | 0/3 [00:06<?, ?it/s]


ValueError: model.embed_tokens.weight doesn't have any device set.

In [ ]:
pipe = pipeline('text-generation',
                model=model,
                tokenizer=tokenizer,
                max_length=256,
                temperature=0.6,
                top_p=0.95,
                repetition_penalty=1.2)

llm=HuggingFacePipeline(pipeline=pipe)

template = """Below is an instruction that describes a task. Write a response that appropriately completes

### Instruction:
{instruction}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

llm_chain = LLMChain(prompt=prompt,
                     llm=llm)

question = "What is the capital of India?"

print(llm_chain.run(question))